# Réseau de Neurones pour la Classification des Galaxies

Ce carnet entraîne un réseau de neurones convolutionnel pour classifier les galaxies en utilisant le dataset Galaxy10 DECals. Il explore différents optimisateurs et taux d'apprentissage pour trouver la meilleure combinaison.

In [ ]:
# Importer les bibliothèques nécessaires
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from PIL import Image
from datetime import datetime

## Prétraitement des Données

La fonction suivante prétraite le dataset en redimensionnant les images à 64x64 pixels et en les normalisant. Le dataset est ensuite divisé en ensembles d'entraînement et de test.

In [ ]:
def preprocess_data(dataset):
    new_size = (64, 64)
    images = np.array([np.array(Image.fromarray(np.array(image['image'])).resize(new_size)) for image in dataset['train']])
    labels = np.array(dataset['train']['label'])

    images = images / 255.0

    return train_test_split(images, labels, test_size=0.2, random_state=42)

## Construction du Modèle

La fonction suivante construit un modèle de réseau de neurones convolutionnel avec les paramètres spécifiés.

In [ ]:
def build_model(layer1_neuron, layer2_neuron, layer3_neuron, layer4_neuron, lr, kernel_size, activation_function, dropout_value, opt_name):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(64, 64, 3)),
        tf.keras.layers.Conv2D(layer1_neuron, kernel_size, activation=activation_function),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(layer2_neuron, kernel_size, activation=activation_function),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(layer3_neuron, kernel_size, activation=activation_function),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(layer4_neuron, activation=activation_function),
        tf.keras.layers.Dropout(dropout_value),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer=opt_name(learning_rate=lr),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

## Paramètres

Nous définissons ici les paramètres pour le réseau de neurones, les optimiseurs, et les taux d'apprentissage.

In [ ]:
# Paramètres
nb_epochs = 15  # Nombre d'époques
layer1_neuron = 32
layer2_neuron = 64
layer3_neuron = 128
layer4_neuron = 64

kernel_size = (3, 3)
activation_function = 'relu'
dropout_value = 0.5

optimizers = {
    # 'SGD': tf.keras.optimizers.SGD,
    # 'Adagrad': tf.keras.optimizers.Adagrad,
    'Adam': tf.keras.optimizers.Adam
}

learning_rates = [1e-3]#, 1e-2, 1e-1]

## Chargement du Dataset

Charger le dataset Galaxy10 DECals et le prétraiter.

In [ ]:
print("\nChargement du Dataset...\n")
dataset = load_dataset("matthieulel/galaxy10_decals")
train_images, test_images, train_labels, test_labels = preprocess_data(dataset)

## Entraînement et Évaluation

Itérer à travers chaque combinaison d'optimiseur et de taux d'apprentissage, entraîner le modèle et évaluer ses performances. TensorBoard est utilisé pour surveiller le processus d'entraînement.

In [ ]:
best_opt = ''
best_lr = 0
best_acc = 0

print("\nItération sur chaque optimiseur et chaque taux d'apprentissage\n")
for opt_name, opt_class in optimizers.items():
    for lr in learning_rates:
        # Créer un répertoire de logs
        log_dir = f"runs/{opt_name}_lr_{lr}_" + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        print(f"\nCréation du fichier {log_dir}\n")

        # Entraînement et test du réseau de neurones
        print("\nEntraînement...\n")
        model = build_model(layer1_neuron, layer2_neuron, layer3_neuron, layer4_neuron, lr, kernel_size, activation_function, dropout_value, opt_class)
        model.fit(train_images, train_labels, epochs=nb_epochs, validation_data=(test_images, test_labels), callbacks=[tensorboard_callback])
        test_loss, test_acc = model.evaluate(test_images, test_labels)
        if best_acc < test_acc:
            best_acc = test_acc
            best_lr = lr
            best_opt = opt_name
        print(f"Optimiseur: {opt_name}, lr: {lr} - Précision sur le test: {test_acc*100:.2f}%\n")

print(f"\nMeilleure combinaison: (Optimiseur: {best_opt} - Taux d'Apprentissage: {best_lr} - Précision: {best_acc*100:.4f})\n")

## Exécution de TensorBoard

Pour surveiller le processus d'entraînement, exécutez la commande suivante dans votre terminal :
```bash
tensorboard --logdir=runs
```
Ensuite, ouvrez un navigateur web et allez à l'URL fournie par TensorBoard (généralement `http://localhost:6006`).